In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical, Sequence
import albumentations as A

# Define Albumentations transform
transform = A.Compose([
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.CLAHE(clip_limit=2.0, p=0.5)
    ], p=0.4),
    A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=10, p=0.3),
    A.GaussNoise(var_limit=(5, 15), p=0.1),
    A.ImageCompression(quality_lower=85, quality_upper=100, p=0.2),
    A.Equalize(p=0.1),
    A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.02, rotate_limit=9, border_mode=0, p=0.5),
    A.MotionBlur(blur_limit=(3, 5), p=0.1),
    A.Resize(64, 64)
])


2025-07-11 09:06:01.409599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752224761.603974      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752224761.665790      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_36/781083476.py:15: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5, 15), p=0.1),
/tmp/ipykernel_36/781083476.py:16: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=85, quality_upper=100, p=0.2),
/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: Use

In [4]:
from glob import glob

dataset_path = "/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
class_names = sorted(os.listdir(dataset_path))
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(class_names)}

image_paths = []
labels = []

for class_name in class_names:
    folder = os.path.join(dataset_path, class_name)
    for img_file in glob(os.path.join(folder, "*.jpg")):
        image_paths.append(img_file)
        labels.append(class_to_idx[class_name])

# Split into train/val (80/20)
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)


In [11]:
class AlbumentationsGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size, transform, num_classes, augmentations_per_image=4):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.transform = transform
        self.num_classes = num_classes
        self.augmentations_per_image = augmentations_per_image

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_paths = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        images, labels = [], []

        for img_path, label in zip(batch_paths, batch_labels):
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            for _ in range(self.augmentations_per_image):
                augmented = self.transform(image=image)['image']
                images.append(augmented / 255.0)  # normalize
                labels.append(label)

        return np.array(images), to_categorical(labels, num_classes=self.num_classes)


In [12]:
batch_size = 64
num_classes = len(class_names)

train_gen = AlbumentationsGenerator(
    train_paths, train_labels, batch_size=batch_size,
    transform=transform, num_classes=num_classes,
    augmentations_per_image=9
)

val_gen = AlbumentationsGenerator(
    val_paths, val_labels, batch_size=batch_size,
    transform=transform, num_classes=num_classes,
    augmentations_per_image=1  # usually no strong augmentation in validation
)


In [13]:
CLASSES = [
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
    'del','nothing','space'
]

CONFIGURATION = {
    "BATCH_SIZE": 64,
    "IM_SIZE": 64,
    "LEARNING_RATE": 0.001,
    "NUM_CLASSES": 29,
}

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()

# Convolutional Block 1
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(CONFIGURATION['IM_SIZE'], CONFIGURATION['IM_SIZE'], 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Convolutional Block 2
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Convolutional Block 3
model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Convolutional Block 4
model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Flatten and Dense Layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# Output Layer
model.add(Dense(29, activation='softmax'))  # 29 classes: A-Z + space, del, nothing


In [ ]:

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import cv2
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

model.fit(train_gen, validation_data=val_gen, epochs=21, callbacks=callbacks)

Epoch 1/21
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1662s 2s/step - accuracy: 0.2032 - loss: 2.6471 - val_accuracy: 0.9270 - val_loss: 0.2096 - learning_rate: 0.0010
Epoch 2/21
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1280s 1s/step - accuracy: 0.8926 - loss: 0.3276 - val_accuracy: 0.9860 - val_loss: 0.0499 - learning_rate: 0.0010
Epoch 3/21
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1256s 1s/step - accuracy: 0.9532 - loss: 0.1685 - val_accuracy: 0.9930 - val_loss: 0.0282 - learning_rate: 0.0010
Epoch 4/21
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1217s 1s/step - accuracy: 0.9690 - loss: 0.1175 - val_accuracy: 0.9961 - val_loss: 0.0185 - learning_rate: 0.0010
Epoch 5/21
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1202s 1s/step - accuracy: 0.9745 - loss: 0.0983 - val_accuracy: 0.9967 - val_loss: 0.0146 - learning_rate: 0.0010
Epoch 6/21
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1201s 1s/step - accuracy: 0.9776 - loss: 0.0890 - val_accuracy: 0.9961 - val_loss: 0.0181 - learning_rate: 0.0010
Epoch 7/21
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1204s 1s/step - accura